# 0. Visual Search - Settings

# Visual Search with Azure Cognitive Search, Sentence Transformers, Azure Computer Vision and bar code/QR code detection

## 1. Description
The goal of this is **Azure AI asset is to enable search over Text and Images using Azure Cognitive Search**. The technique was inspired by a research article which show how to **convert vectors (embeddings) to text which allows the Cognitive Search service to leverage the inverted index to quickly find the most relevant items**. For this reason, any model that will convert an object to a vector can be leveraged if the number of dimensions in the resulting vector is less than 3,000. It also allows users to leverage existing pretrained or fine-tuned models.<br><br>

This technique has shown to be incredibly effective and easy to implement. We are using **Sentence Transformers, which is an OpenAI clip model wrapper**. We need to embed all our existing catalog of images. Then the objects embedding are converted into a set of fake terms and all the results are stored into an Azure Cognitive Search index for handling all the search requests.
For example, if an embedding looked like [-0,21, .123, ..., .876], this might be converted to a set of fake terms such as: “A1 B3 … FED0”. This is what is sent as the search query to Azure Cognitive Search.<br><br>

We can **enrich the Azure Cognitive Search index by using extracted text from the images using Azure Read API**. We can also detect and extract any information from **bar code and/or QR code** that might be available in the products catalog images. And we can use also **Azure Computer Vision as well to detect the dominant colors of the image, the tags that can describe the image and the caption of each image**. All these information will be ingested into the Azure Cognitive Search index.<br><br>

The goal of this asset is to be able to use the inverted index within Azure Cognitive Search to be able to quickly find vectors stored in the search index that are like a vector provided as part of a search query and/or using any AI extracted information (text, dominant colors, …). Unlike techniques like cosine similarity which are slow to process large numbers of items, this leverages an inverted index which enables much more data to be indexed and searched.<br>

## 2. Process

- We have here a collection of catalog images (466 images).
- For each of these images, we will embed them using Sentence Transformers.  Sentence Transformer can be used to map images and texts to the same vector space. As model, we use the OpenAI CLIP Model which was trained on a large set of images and image alt texts.
- We can retrieve any text from these images using Azure Read API (if any text is available)
- We can retrieve any text information from any bar code or QR code (if any)
- All these information will be ingested into an Azure Cognitive Search index
- Then if you have a field image, you can embed it and extract any text/barcode information and call the Azure Cognitive Search index to retrieve any similar images using vecText similarity and/or using any query text from the extracted text


<img src="https://github.com/retkowsky/images/blob/master/process.png?raw=true">

Field images are available in the field images directory (number of images=53)


## 3. Azure products documentation
- https://azure.microsoft.com/en-us/products/search/ 
- https://azure.microsoft.com/en-us/products/cognitive-services/computer-vision/#overview 
- https://learn.microsoft.com/en-us/azure/cognitive-services/Computer-vision/how-to/call-read-api 
- https://zbar.sourceforge.net/ 
- https://github.com/liamca/vector-search

## 4. Research article
https://www.researchgate.net/publication/305910626_Large_Scale_Indexing_and_Searching_Deep_Convolutional_Neural_Network_Features
    
## 5. Directories
- **images**: We have two directories (catalog images, field images)
- **model**: Directory to save the clusters of the model
- **results**: Directory to save some results
- **test**: Directory that contains some testing images

## 6. Python notebooks

### 0. Settings.ipynb
Notebook that contains the link to the images and the importation process of the python required libraries

### 1. Catalog images exploration.ipynb
This notebook will display some catalog and field images

### 2.OpenAI Clip and VecText Clusters.ipynb
This notebook will explain what sentence transformers is and will generate the clusters
This notebook analyzes a set of existing images to determine a set of "cluster centers" that will be used to determine which "fake words" are generated for a vector
This notebook will take a test set of files (testSamplesToTest) and determine the optimal way to cluster vectors into fake words that will be indexed into Azure Cognitive Search

### 3. VecText generation.ipynb
This notebook will generate the vectext embedding for all the catalog images

### 4. BarCode Information extraction.ipynb
This notebook will detect any barcode or QR code from the catalog images and will extract the information

### 5. Azure CV for OCR, tags, colors and captions.ipynb
This notebook will use Azure Computer Vision or OCR, colors, tags and caption extraction for each of the catalog images.

### 6. Azure Cognitive Search Index Generation.ipynb
This notebook will show how to ingest all the information into an Azure Cognitive Search index.

### 7. Calling Azure Cognitive Search.ipynb
We can now test the index using some images similarity visual search or free text queries using azure Cognitive Search.

## 7. Python files

- **azureCognitiveSearch.py**
This python file contains many functions to manage and use Azure Cognitive Search

- **myfunctions.py**
This python file contains many generic functions used in all the notebooks

- **vec2Text.py**
This python file contains some functions for the sentence transformers model


18-Oct-2022 Serge Retkowsky | serge.retkowsky@microsoft.com | https://www.linkedin.com/in/serger/

In [1]:
import myfunctions as my
import sys

In [2]:
sys.version

'3.8.5 (default, Sep  4 2020, 07:30:14) \n[GCC 7.3.0]'

## 1. azureCognitiveSearch.py
> This is a custom .py file with some Azure Cognitive Search functions

In [3]:
my.view_file("azureCognitiveSearch.py")

Viewing file: azureCognitiveSearch.py 

# Azure Cognitive Search python functions
# 18-Oct-2022


# Multiple imports
import os
import configparser
import requests
import json
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import time
import vec2Text

import myfunctions as my

from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents import SearchClient
from azure.search.documents.indexes.models import (
    ComplexField,
    CorsOptions,
    SearchIndex,
    ScoringProfile,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
)
from IPython.display import Image

# Azure Cognitive Search connection
config_file = 'azureconfig.ini'

# Azure Cognitive Search Index name
index_name = "demo-retail"

# Azure Cognitive Search credentials
config = configparser.ConfigParser()
config.read(config_file)
acs_key = config.get('AzureCognitiveSearch', 'key')
acs_endpoint = config.get('AzureCo

## 2. myfunctions.py
> This is a custom .py file with some generic python functions

In [4]:
my.view_file("myfunctions.py")

Viewing file: myfunctions.py 

# Generic python functions
# Serge Retkowsky (serge.retkowsky@microsoft.com)
# 18-Oct-2022


# Multiple imports
import configparser
import cv2
import datetime
import glob
import humanize
import json
import logging
from math import *
import numpy as np
import os
import pathlib
import pendulum
import platform
import psutil
import random
import re
import requests
import shutil
import sys
import socket
import time
import uuid

from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials
from matplotlib import pyplot as plt
from pyzbar.pyzbar import decode

# Reading config.ini file to retrieve credentials
config = configparser.ConfigParser()
config_file = 'azureconfig.ini'
config.read(config_file)

# Azure CV Cre

## 3. Azure CV credentials

You need to update the azureconfig.ini file

In [5]:
!ls azureconfig.ini -lh

-rwxrwxrwx 1 root root 413 Oct 18 12:36 azureconfig.ini


## 4. Python librairies installation

In [6]:
#!pip install kaleido

In [7]:
#!pip install --upgrade azure-cognitiveservices-vision-computervision

In [8]:
# To install pyzbar:
#!sudo apt-get install libzbar0 --yes
#!pip install pyzbar

In [9]:
#!conda create -n py37-vector-search python=3.7 --yes

In [10]:
#!conda install -c conda-forge jupyterlab
#!conda install pytorch torchvision torchaudio cpuonly -c pytorch --yes

In [11]:
#!conda install -c anaconda gensim --yes

In [12]:
#!conda install smart_open==2.0.0 --yes

In [13]:
#!pip install azure-search-documents

In [14]:
#!pip install -U sentence-transformers

In [15]:
#pip install ftfy